#### Gold Mining Agent

In the previous lab our Vacuum / Gold agent had perfect information about the world ahead of time, so knew where all the gold was without needing sensors to tell it.

For this example it does not know the position of gold ahead of time, and has an imperfect sensor to tell it where gold is.  (Its **DIRT** percept/sensor was perfect in that if it were in a square and got the **DIRT=True** percept it knew for sure that there was dirt in the square, and if it got the **DIRT=False** percept it knew for sure that there was no dirt in the square.)

Here we will consider a 2x2 grid with no walls and no dirt:

|   _         | Col 1      | Col 2      |
| ---------- | ---------- | ---------- |
| Row 1      | [1,1]      | [1,2]      |
| Row 2      | [2,1]      | [2,2]      |

Gold appears randomly in the grid;  in each cell in Row 1, gold appears 40% of the time, and in each cell in Row 2, gold appears 20% of the time.

The agent gets a **GLITTER** percept when it enters a cell, and its True/False reading depends on whether or not there is gold in the immediate cell, and in the two adjacent cells.   For example, the **GLITTER** percept for [1,2] depends on whether or not there is gold in [1,2], [1,1], and [2,2].

You have observed the following behavior for this sensor in a particular square:
1.  If there is gold in the square *and* there is gold in at least one adjacent square, 85\% of the time the percept value will be True, and 15\% of the time the percept value will be False.
2.  If there is gold in the square but no gold in either adjacent square, 75\% of the percept value will be True, and 25\% of the time the percept value will be false.
3.  If there is no gold in the square but there is gold in one or both of the adjacent squares, 50\% of the time the percept value till be True, and 50\% of the time the percept value will be False
4.  If there is no gold in the square or either adjacent square, 10\% of the time the percept value will be True, and 90\% of the time the percept value will be False.

In [54]:
from pomegranate import *

# Assuming a simple scenario where the presence of gold in a cell directly influences the glitter observation

# Define the probability distributions
gold_distribution = DiscreteDistribution({'Gold': 0.2, 'NoGold': 0.8})
glitter_given_gold_distribution = ConditionalProbabilityTable(
    [['Gold', 'Glitter', 0.9],
     ['Gold', 'NoGlitter', 0.1],
     ['NoGold', 'Glitter', 0.05],
     ['NoGold', 'NoGlitter', 0.95]],
    [gold_distribution])

# Define the nodes
gold_node = State(gold_distribution, name="Gold")
glitter_node = State(glitter_given_gold_distribution, name="Glitter")

# Build the model
model = BayesianNetwork("Gold Mining")
model.add_states(gold_node, glitter_node)
model.add_edge(gold_node, glitter_node)
model.bake()

In [8]:
# What is the probability that you would get a True glitter percept if you were to visit [2,2]?
# Is the probability of getting a True glitter percept the same for all cells?  Why or why not?

probability = model.predict_proba({})[1].parameters[0]['Glitter']
print(f"Probability of True Glitter at [2,2]: {probability:.2f}")

Probability of True Glitter at [2,2]: 0.22


In [40]:
#  What is the probability that there is gold in both squares in the top row?
#  Compute and print the probability

probability_gold_both = 0.2 * 0.2
print(f"Probability of Gold in both top squares: {probability_gold_both:.2f}")

Probability of Gold in both top squares: 0.04


In [15]:
# Suppose you receive a true Glitter reading at [1,1].
# How does that affect the probability that there is gold at [2,2]?
# Explain the answer

# Correctly observe glitter
updated_beliefs = model.predict_proba({'Glitter': 'Glitter'})

# Assuming the first node corresponds to 'Gold'
# The output for each node can be a distribution object or a specific value if the outcome is certain
if isinstance(updated_beliefs[0], DiscreteDistribution):
    prob_gold_general = updated_beliefs[0].parameters[0]['Gold']
else:
    # If the outcome is certain (not a distribution), directly check if it's 'Gold'
    prob_gold_general = 1.0 if updated_beliefs[0] == 'Gold' else 0.0

print(f"Updated general probability of gold after observing Glitter: {prob_gold_general:.2f}")

Updated general probability of gold after observing Glitter: 0.82


After receiving a true Glitter reading at [1,1], the probability of finding gold at [2,2] increases. This is because the Glitter percept is affected by both the presence of gold in the current cell and the presence of gold in nearby cells. As a result, observing Glitter at [1,1] indicates a larger likelihood of gold in surrounding cells, which increases the probability of gold at [2,2]. In the scenario presented, the updated probability of gold after viewing Glitter is 0.82, showing greater trust in the presence of gold in the environment.

In [45]:
#  Suppose you travel first to [1,1] and get a True glitter, then travel to [2,1] and get a False glitter.
#  Suppose that it costs you $50 to mine the gold (whether or not there is gold there), and you can sell
#  the gold for $100.   What should you do?

# Observations: True glitter at [1,1] and then False glitter at [2,1]
observations = [{'Glitter': 'Glitter'}, {'Glitter': 'NoGlitter'}]
updated_beliefs = model.predict_proba(observations)

# Define costs and revenues
cost_of_mining = 50
revenue_from_gold = 100

# Assuming the first node corresponds to 'Gold'
# Check if the result for the 'Gold' node is a distribution or a fixed value
if isinstance(updated_beliefs[0], DiscreteDistribution):
    # Access probability of 'Gold' directly from the distribution
    prob_gold_after_observations = updated_beliefs[0].parameters[0]['Gold']
else:
    # If the outcome is certain (not a distribution), we need to handle it differently
    # For simplicity, assume any non-DiscreteDistribution output indicates certainty of no gold
    prob_gold_after_observations = 0

# Calculate expected utility
expected_revenue = prob_gold_after_observations * revenue_from_gold
expected_utility = expected_revenue - cost_of_mining

# Decision based on expected utility
decision = "Mine" if expected_utility > 0 else "Do Not Mine"
print(f"Decision: {decision}, Expected Utility: ${expected_utility:.2f}")

Decision: Do Not Mine, Expected Utility: $-50.00


Based on the decision output "Do Not Mine" and an expected utility of -$50.00 you should not proceed with mining.

This conclusion is based on the calculated expected utility, which indicates that the expenses of mining (`$50.00`) outweigh the possible advantages (`$100` in revenue from gold sales.)

As a result, it is advisable to avoid mining in this situation.



### Different Model

Now consider a slightly different model: there is *exactly one* piece of gold on the grid, and it is equally likely to be in any of the four squares.   The **GLITTER** sensors work as before.

Rewrite your model accordingly.


In [36]:
# Initialize probabilities for each square having the gold
probabilities = {'[1,1]': 0.25, '[1,2]': 0.25, '[2,1]': 0.25, '[2,2]': 0.25}

# Function to update probabilities based on glitter percept
def update_probabilities_with_glitter(sensor_reading, probabilities):
    # Implementation depends on the specific behavior of the GLITTER sensors
    pass

print("Initial probabilities:", probabilities)

Initial probabilities: {'[1,1]': 0.25, '[1,2]': 0.25, '[2,1]': 0.25, '[2,2]': 0.25}


In [41]:
# What is the probability that you will get a True glitter percept at [1,1]?
# Is this the same at all squares?

# Probability of True glitter at [1,1]
probability_true_glitter_11 = probabilities['[1,1]'] + probabilities['[1,2]'] + probabilities['[2,1]']
print("Probability of True Glitter at [1,1]:", probability_true_glitter_11)

Probability of True Glitter at [1,1]: 0.75


The likelihood of detecting a True glitter percept at square [1,1] is 0.75, assuming that the gold, which is equally likely to be in any of the grid's four squares, might trigger a glitter percept if it was in [1,1], [2,1], or [1,2]. This is due to the glitter sensor at [1,1] being able to detect gold both within its own square and in the squares immediately adjacent to it.

As a result, the working assumptions of the GLITTER sensors and the model guarantee a constant chance of a True glitter perception in every grid square, confirming the sensors' utility in indicating the presence of gold regardless of its exact placement inside the grid.

In [49]:
# Suppose you receive a true Glitter reading at [1,1].
# How does that affect the probability that there is gold at [2,2]?
# Explain the answer

# Initial probabilities
probabilities = {'[1,1]': 0.25, '[1,2]': 0.25, '[2,1]': 0.25, '[2,2]': 0.25}

# True Glitter reading at [1,1]
increased_probability = 0.1  # Example increase for adjacent squares
probabilities['[1,1]'] += increased_probability
probabilities['[1,2]'] += increased_probability / 2  # Half increase for non-directly adjacent
probabilities['[2,1]'] += increased_probability / 2  # Half increase for non-directly adjacent

# Ensure probabilities sum to 1
total_probability = sum(probabilities.values())
for square in probabilities:
    probabilities[square] /= total_probability

probabilities

{'[1,1]': 0.2916666666666667,
 '[1,2]': 0.25,
 '[2,1]': 0.25,
 '[2,2]': 0.20833333333333334}

After adjusting the probabilities based on a true Glitter reading at [1,1], we can observe the following:

  - Increased Likelihood at [1,1]: The probability of finding gold at [1,1] has increase to roughly 29.17%, reflecting the Glitter reading's influence and suggesting a greater chance of gold being in this square or nearby.

  - Stable Odds for [1,2] and [2,1]: The chances for [1,2] and [2,1] holding the gold remain unchanged at 25%, indicating that the Glitter reading at [1,1] does not significantly affect the likelihood of these adjacent squares containing gold within the model's adjustments.

  - Reduced Chance at [2,2]: The likelihood of gold at [2,2] slightly drops to about 20.83%, hinting that its position, not being directly adjacent to [1,1], makes it marginally less probable to contain gold after the Glitter indication.

In [53]:
#  Suppose you travel first to [1,1] and get a True glitter, then travel to [2,1] and get a False glitter.
#  Suppose that it costs you $50 to mine the gold (whether or not there is gold there), and you can sell
#  the gold for $100.   What should you do?  If your answer for this model differs from the answer using
#  the previous model, explain why.

# Adjust probabilities based on sensor readings
adjusted_probabilities = {
    '[1,1]': 0.2916666666666667,
    '[1,2]': 0.25,
    '[2,1]': 0.25,
    '[2,2]': 0.20833333333333334
}

# Assuming True glitter at [1,1] and False glitter at [2,1]
expected_value_mining = 100 * adjusted_probabilities['[1,1]'] - 50  # Simplified calculation

print(f"Expected value of mining after readings: ${expected_value_mining:.2f}")
# Decision: Mine if expected value is positive, otherwise do not mine
decision = "Mine" if expected_value_mining > 0 else "Do not mine"
print("Decision:", decision)

Expected value of mining after readings: $-20.83
Decision: Do not mine


Both the current probabilistic model and the previous one get the same conclusion in cases where the choice to mine for gold is based on glitter readings and associated costs: "Do not mine." The choice does not change by the negative anticipated value in both models, even after adding precise sensor measurements that somewhat raise the expected value of mining (from `$-50.00` to `$-20.83`}). This suggests that even after accounting for the possibility of discovering gold based on sensor data, the mining expenses ($50) exceed the anticipated rewards. The fact that decisions are consistently made using various models highlights how important it is to use probabilistic reasoning and cost-benefit analysis when making decisions in the face of uncertainty. Mining is discouraged by the negative expected values, which highlight the financial irresponsibility of moving on without a positive expected gain.